In [ ]:
# 0) Install (safe)
!pip -q install -U tqdm pillow scikit-learn

In [ ]:
# 1) Imports & config
import os, random
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm

# ---------- Repro & device ----------
SEED = 1337
def seed_everything(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# ---------- Paths (your repo already cloned) ----------
# If you haven't cloned in this runtime yet, uncomment:
# !git clone https://github.com/Gaabshiine/pycon-2025-hackthon.git
BASE_DIR   = Path('/content/pycon-2025-hackthon')
IMAGES_DIR = BASE_DIR / 'Images'
TRAIN_CSV  = BASE_DIR / 'Train.csv'
TEST_CSV   = BASE_DIR / 'Test.csv'
SAMPLE_SUB = BASE_DIR / 'SampleSubmission.csv'
assert TRAIN_CSV.exists() and TEST_CSV.exists() and IMAGES_DIR.exists()

train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)
print(train_df.head())

# ---------- T4-friendly hparams ----------
IMG_SIZE    = 256          # try 320 if VRAM allows
BATCH_SIZE  = 64           # 48/32 if OOM
NUM_WORKERS = 0            # avoids Colab multiprocessing assertion
PIN_MEMORY  = True

EPOCHS          = 15
LR              = 3e-4
WARMUP_EPOCHS   = 1
WEIGHT_DECAY    = 1e-4
SMOOTH_EPS      = 0.05     # label smoothing

BACKBONE = "resnet18"      # "resnet18" or "resnet34"
FOLDS    = 3               # 3-fold CV ensemble

# ---------- AMP (new API) ----------
from contextlib import nullcontext
autocast_ctx = torch.amp.autocast if torch.cuda.is_available() else nullcontext
scaler = torch.amp.GradScaler('cuda') if torch.cuda.is_available() else None

In [ ]:
# 2) Quick EDA (optional)
train_df['Label'].value_counts().sort_index().plot(kind="bar", title="Class balance (0=healthy, 1=fall armyworm)")
plt.show()

In [ ]:
# 3) Dataset & transforms
class MaizeDataset(Dataset):
    def __init__(self, df, images_dir, mode='train', transform=None):
        self.df = df.reset_index(drop=True)
        self.images_dir = Path(images_dir)
        self.mode = mode
        self.transform = transform
        
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.images_dir / row['Image_id']  # CSV already has .jpg
        img = Image.open(img_path).convert('RGB')
        if self.transform: img = self.transform(img)
        if self.mode == 'test':
            return img, row['Image_id']
        else:
            label = torch.tensor(float(row['Label']), dtype=torch.float32)
            return img, label

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
valid_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

In [ ]:
# 4) Model, loss, scheduler
def build_model():
    if BACKBONE == "resnet34":
        m = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
    else:
        m = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    in_feats = m.fc.in_features
    m.fc = nn.Linear(in_feats, 1)
    return m

def bce_with_logits_smooth(logits, labels, eps=SMOOTH_EPS):
    # Smooth labels toward 0.5 to regularize
    labels_s = labels * (1 - eps) + 0.5 * eps
    return F.binary_cross_entropy_with_logits(logits, labels_s)

def make_scheduler(optimizer):
    def lr_lambda(e):
        if e < WARMUP_EPOCHS:      # linear warmup
            return (e + 1) / max(1, WARMUP_EPOCHS)
        t = (e - WARMUP_EPOCHS) / max(1, (EPOCHS - WARMUP_EPOCHS))
        return 0.5 * (1 + np.cos(np.pi * t))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

class EarlyStopper:
    def __init__(self, patience=3): 
        self.best = -np.inf; self.wait = 0; self.patience = patience
    def step(self, val):
        if val > self.best + 1e-4: self.best = val; self.wait = 0; return True
        self.wait += 1; return False
    def should_stop(self): return self.wait >= self.patience

In [ ]:
# 5) Train / Eval loops
def train_one_epoch(model, loader, optimizer, scaler):
    model.train()
    total = 0.0
    for imgs, labels in tqdm(loader, leave=False):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True).unsqueeze(1)
        optimizer.zero_grad(set_to_none=True)
        with autocast_ctx('cuda'):
            logits = model(imgs)
            loss = bce_with_logits_smooth(logits, labels)
        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        total += loss.item() * imgs.size(0)
    return total / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_probs, all_targets = [], []
    for imgs, labels in loader:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True).unsqueeze(1)
        with autocast_ctx('cuda'):
            probs = torch.sigmoid(model(imgs))
        all_probs.append(probs.cpu().numpy())
        all_targets.append(labels.cpu().numpy())
    probs = np.concatenate(all_probs).ravel()
    targs = np.concatenate(all_targets).ravel()
    return roc_auc_score(targs, probs)

In [ ]:
# 6) TTA inference helper
@torch.no_grad()
def predict_tta(model, imgs):
    # orig + hflip + vflip (×3)
    with autocast_ctx('cuda'):
        logits  = model(imgs)
        logits += model(torch.flip(imgs, [3]))
        logits += model(torch.flip(imgs, [2]))
    return torch.sigmoid(logits / 3.0)

In [ ]:
# 7) 3-fold CV training + ensemble submission
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

test_ds = MaizeDataset(test_df, IMAGES_DIR, 'test', valid_tfms)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                         num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

test_preds = np.zeros(len(test_df), dtype=np.float32)

for fold, (tr_idx, va_idx) in enumerate(skf.split(train_df['Image_id'], train_df['Label']), 1):
    print(f"\n========== Fold {fold}/{FOLDS} ==========")
    tr_df = train_df.iloc[tr_idx].reset_index(drop=True)
    va_df = train_df.iloc[va_idx].reset_index(drop=True)

    tr_ds = MaizeDataset(tr_df, IMAGES_DIR, 'train', train_tfms)
    va_ds = MaizeDataset(va_df, IMAGES_DIR, 'train', valid_tfms)

    tr_loader = DataLoader(tr_ds, batch_size=BATCH_SIZE, shuffle=True,
                           num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
    va_loader = DataLoader(va_ds, batch_size=BATCH_SIZE, shuffle=False,
                           num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

    model = build_model().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = make_scheduler(optimizer)
    early = EarlyStopper(patience=3)

    best_path = BASE_DIR / f'best_{BACKBONE}_fold{fold}.pt'
    best_auc = -1

    for epoch in range(1, EPOCHS + 1):
        loss = train_one_epoch(model, tr_loader, optimizer, scaler)
        val_auc = evaluate(model, va_loader)
        print(f"Fold {fold} | Epoch {epoch:02d} | loss {loss:.4f} | val_auc {val_auc:.6f}")
        if early.step(val_auc):
            torch.save(model.state_dict(), best_path)
            best_auc = val_auc
            print("  ✓ Saved best")
        scheduler.step()
        if early.should_stop():
            print("  Early stopping.")
            break

    # Load best & predict test
    model.load_state_dict(torch.load(best_path, map_location=device))
    model.eval()
    fold_probs = []
    for imgs, _ids in tqdm(test_loader, leave=False):
        imgs = imgs.to(device, non_blocking=True)
        p = predict_tta(model, imgs).cpu().numpy().ravel()
        fold_probs.append(p)
    fold_probs = np.concatenate(fold_probs)
    test_preds += fold_probs / FOLDS
    print(f"Fold {fold} best AUC: {best_auc:.6f}")

# Save ensemble submission
sub = pd.DataFrame({'Image_id': test_df['Image_id'], 'Label': test_preds})
sub_path = BASE_DIR / f'submission_cv{FOLDS}_{BACKBONE}_img{IMG_SIZE}.csv'
sub.to_csv(sub_path, index=False)
print(sub.head(), "\nSaved:", sub_path)

In [ ]:
from google.colab import files
files.download(str(sub_path))